### Importing libraries

In [20]:
!pip install opencv-python

  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/ec/6c/fab8113424af5049f85717e8e527ca3773299a3c6b02506e66436e19874f/opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.1/38.8 MB 1.7 MB/s eta 0:00:24
   ---------------------------------------- 0.2/38.8 MB 2.9 MB/s eta 0:00:14
    --------------------------------------- 0.5/38.8 MB 4.1 MB/s eta 0:00:10
   - -------------------------------------- 1.0/38.8 MB 5.6 MB/s eta 0:00:07
   - -------------------------------------- 1.4/38.8 MB 6.4 MB/s eta 0:00:06
   - -------------------------------------- 1.8/38.8 MB 6.9 MB/s eta 0:00:06
   - -------------------------------------- 1.9/38.8 MB 7.0 MB/s eta 0:00:06
   - -------------------------------------- 1.9/38.8 MB 7.0 MB/s eta 0:00:06
   - -------------------------------------- 1.9/38.8 MB 5.3 MB/s eta 0:00:07
   --- --------

In [39]:
import tensorflow as tf
import os
import random
import cv2
import numpy as np

### Dataset

Il dataset contiene 100k training images divise per 200 classi. Ciò che vedrete, scaricando il dataset, saranno 3 folders (train, test e validation) e due file di testo che contengono informazioni generiche sui nomi delle cartelle e sul tipo di classe che rappresentano (persone, entità, ecc.). La train folder contiene altre 200 cartelle, corrispondenti alle 200 classi, che contengono 500 immagini ognuna. I file nomecartella_boxes.txt che vedete contengono informazioni sulle bounding box presenti in ciascuna immagine (coordinate x e y del top-left corner, width e height). Validation e test contengono 10k immagini ognuna e sono shufflate randomicamente.

Purtroppo il dataset non si trova in nessuna libreria python, quindi va necessariamente scaricato sul vostro pc. Vi lascio il link: http://cs231n.stanford.edu/tiny-imagenet-200.zip 

Non vi preoccupate se vi dice che il sito non è sicuro, voi scaricatelo comunque. Per ora il computer non mi è esploso quindi credo vada bene.

#### Training set

In [31]:
# Cambiate il path a seconda di dove mettete il dataset
# La r non è un errore: serve ad evitare l'unicode error
dataset_dir = r"C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\train"  
train_data = []

for folder_name in os.listdir(dataset_dir): # entra semplicemente in ciascuna delle 200 folder
    folder_path = os.path.join(dataset_dir, folder_name) 
    
    if os.path.isdir(folder_path): 
        images_folder_path = os.path.join(folder_path, 'images')  
        # in ciascuna folder, c'è una cartella images e un file .txt che mi stava portando casini: con questa riga di codice
        # vado a considerare solo le cartelle images e ad ignorare il file di testo
        
        if os.path.isdir(images_folder_path): 
            # ora sono dentro la cartella images (iterata sempre sulle 200 folders)
            for img_name in os.listdir(images_folder_path):
                img_path = os.path.join(images_folder_path, img_name)
                
                if os.path.isfile(img_path) and (img_path.endswith('.JPEG')):
                    try:
                        img = cv2.imread(img_path)

                        if img is not None:  
                            train_data.append(img)
                    except Exception as e:
                        print(f"Error loading image {img_path}: {e}")

# Sto usando numpy invece di torch ma credo vada bene comunque: ho controllato e ciascuna immagine è 64x64x3 quindi dovrebbe 
# andar bene. Voi controllate per sicurezza con le righe che commento sotto
train_data = np.array(train_data)
# train_data[:1].shape (dovrebbe uscire una roba del tipo (1, 64, 64, 3) perchè le immagini sono RGB e 64x64 come dice il sito)
# len(train_data) # dovrebbe uscire 100k perchè, come detto prima, sono 100k immagini

Questo pezzo di codice, su jupyter anaconda, impiega circa 10 minuti.

#### Validation e test

In [45]:
validation_dir = r"C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\val\images"   
test_dir = r"C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\test\images" 
# state attente ad inserire \images alla fine se no vi considera pure il file di testo

validation_data = []
test_data = []

def load_images_from_directory(directory):
    data = []
    for img_name in os.listdir(directory):
        img_path = os.path.join(directory, img_name)
        if os.path.isfile(img_path) and (img_path.endswith('.JPEG')):
            try:

                img = cv2.imread(img_path)

                if img is not None:  
                    data.append(img)
            except Exception as e:
                print(f"Error loading image {img_path}: {e}")
    
    return data


validation_data = load_images_from_directory(validation_dir)
test_data = load_images_from_directory(test_dir)


validation_data = np.array(validation_data)
test_data = np.array(test_data)

print(f"Loaded {len(validation_data)} validation images.")
print(f"Loaded {len(test_data)} test images.")
# se tutto è andato bene, dovrebbe darvi 10k per entrambi 
print(validation_data[:1].shape, test_data[:1].shape) # dovrebbe risultare come nella cella sopra

C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\val\images\val_0.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\val\images\val_1.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\val\images\val_10.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\val\images\val_100.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\val\images\val_1000.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\val\images\val_1001.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\val\images\val_1002.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\val\images\val_1003.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\val\images\val_1004.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\val\images\val_1005.JPEG
C:\Users\bongi\OneD

C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\val\images\val_1094.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\val\images\val_1095.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\val\images\val_1096.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\val\images\val_1097.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\val\images\val_1098.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\val\images\val_1099.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\val\images\val_11.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\val\images\val_110.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\val\images\val_1100.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\val\images\val_1101.JPEG
C:\Users\bong

C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\val\images\val_1190.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\val\images\val_1191.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\val\images\val_1192.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\val\images\val_1193.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\val\images\val_1194.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\val\images\val_1195.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\val\images\val_1196.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\val\images\val_1197.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\val\images\val_1198.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\val\images\val_1199.JPEG
C:\Users\b

C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\val\images\val_1276.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\val\images\val_1277.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\val\images\val_1278.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\val\images\val_1279.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\val\images\val_128.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\val\images\val_1280.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\val\images\val_1281.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\val\images\val_1282.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\val\images\val_1283.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\val\images\val_1284.JPEG
C:\Users\bo

C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\val\images\val_1379.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\val\images\val_138.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\val\images\val_1380.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\val\images\val_1381.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\val\images\val_1382.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\val\images\val_1383.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\val\images\val_1384.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\val\images\val_1385.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\val\images\val_1386.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\val\images\val_1387.JPEG
C:\Users\bo

...

C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\test\images\test_9573.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\test\images\test_9574.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\test\images\test_9575.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\test\images\test_9576.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\test\images\test_9577.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\test\images\test_9578.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\test\images\test_9579.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\test\images\test_958.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\test\images\test_9580.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\test\images\test_9

C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\test\images\test_9659.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\test\images\test_966.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\test\images\test_9660.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\test\images\test_9661.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\test\images\test_9662.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\test\images\test_9663.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\test\images\test_9664.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\test\images\test_9665.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\test\images\test_9666.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\test\images\test_9

C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\test\images\test_9758.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\test\images\test_9759.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\test\images\test_976.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\test\images\test_9760.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\test\images\test_9761.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\test\images\test_9762.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\test\images\test_9763.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\test\images\test_9764.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\test\images\test_9765.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\test\images\test_9

C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\test\images\test_9841.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\test\images\test_9842.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\test\images\test_9843.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\test\images\test_9844.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\test\images\test_9845.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\test\images\test_9846.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\test\images\test_9847.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\test\images\test_9848.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\test\images\test_9849.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\test\images\test_

C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\test\images\test_9930.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\test\images\test_9931.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\test\images\test_9932.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\test\images\test_9933.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\test\images\test_9934.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\test\images\test_9935.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\test\images\test_9936.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\test\images\test_9937.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\test\images\test_9938.JPEG
C:\Users\bongi\OneDrive\Desktop\computer_vision\project\tiny-imagenet-200\test\images\test_